In [ ]:
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
from warnings import filterwarnings
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, ShuffleSplit, GridSearchCV 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
import seaborn as sns 

In [ ]:
train_data = pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
train_data

In [ ]:
train_data.info()

In [ ]:
train_data.drop(['id'], 1, inplace=True)

In [ ]:
def change_object_to_int(self):
    data = self.copy()
    o_columns = data.select_dtypes(include=object).columns
    for column in o_columns:
        le = LabelEncoder()
        le.fit(data[column])
        data[column] = le.transform(data[column])
        print(le.classes_)
    return data

In [ ]:
change_object_to_int(train_data)

In [ ]:
tr_data = change_object_to_int(train_data)

In [ ]:
mapping={0:1,1:0, 2:2}
tr_data['Vehicle_Age'] = tr_data['Vehicle_Age'].map(mapping)  # 順序をただす

In [ ]:
tr_data

In [ ]:
def visualize_test_of_no_correlation(self):
    corrPearson = self.corr(method='pearson')
    corrSpearman = self.corr(method='spearman')
    corr_list = [corrPearson, corrSpearman]
    names = ['Pearson', 'Spearman']
    for i, corr in enumerate(corr_list):
        plt.figure(figsize=(10,8))
        sns.heatmap(corr,annot=True, cmap='RdYlGn', vmin=-1, vmax=+1)
        plt.title('{}'.format(names[i]), fontsize=18, color='blue')
        plt.xlabel('columns', fontsize=18)
        plt.ylabel('columns', fontsize=18)
        plt.show()

In [ ]:
visualize_test_of_no_correlation(tr_data)

In [ ]:
tr_data.corr(method='spearman') > 0.8

In [ ]:
y = tr_data['Response']
x = tr_data.drop(['Response'], 1)

In [ ]:
ols = sm.OLS(y, x).fit()
ols.summary()

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.2, random_state=4)

In [ ]:
x_train

In [ ]:
y_train.sum() / y_train.count(), y_test.sum() / y_test.count(), y.sum() / y.count()
# よって、平等に割り振られているがこれではちゃんとしたモデルができないので、0.5になるように割り振る

In [ ]:
def create_appropriate_data(self, t_column):
    df = pd.DataFrame()
    true = self[self[t_column] == 1]
    true_num = len(true)
    false = self[self[t_column] == 0].sample(frac=1) #.reset_index(drop=True)
    fal = false.iloc[:true_num]
    df = df.append(true)
    df = df.append(fal)
    df = df.sample(frac=1)
    return df

In [ ]:
new_train = create_appropriate_data(tr_data, 'Response')

In [ ]:
new_train

In [ ]:
n_train = new_train.reset_index(drop=True)

In [ ]:
n_train

In [ ]:
n_train['Response'].sum() / n_train['Response'].count()
# これで正確なモデルを作ることができる

In [ ]:
n_train.info()

In [ ]:
n_trainV = n_train.copy()

In [ ]:
category_lists = ['Gender', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Response']

In [ ]:
def change_to_category(self, lists):
    for column in lists:
        self[column] = pd.Categorical(self[column])
    return self

In [ ]:
change_to_category(n_trainV, category_lists)

In [ ]:
n_trainV.info()

In [ ]:
#sns.catplot(x='Response', y='Age', data=n_trainV, kind='swarm')   # https://qiita.com/g-k/items/2c5891a27d399645b9aa

In [ ]:
def visualize_bar(self, lists):
    columns = lists[:-1]
    t_column = lists[-1]
    for column in columns:
        try:
            plt.figure(figsize=(15,8))
            #x = self[column]
            #y = self[t_column].sum()
            plt.subplot(1,2,1)
            sns.boxplot(x=column, y=t_column, data=self)
            plt.subplot(1,2,2)
            sns.violinplot(x=column, y=t_column, data=self)
            plt.show()
        except:
            fig = plt.figure(figsize=(15,8))
            ax1 = fig.add_subplot(1,2,1)
            sns.countplot(x=self[self[t_column] == 1][column])
            ax1.set_title('subscribe vehicle insurance', fontsize=14)
            ax2 = fig.add_subplot(1,2,2)
            sns.countplot(x=self[self[t_column] == 0][column])
            ax2.set_title('not subscribe vehicle insurance', fontsize=14)
            fig.show()

In [ ]:
visualize_bar(n_trainV, n_trainV.columns)
# どうしてからの図ができるかはわからない

from above graphs, men tends to be interested in vehicle insurance than women.
And people are getting more interested in as getting older.
and, between vehicle_age from 1 to 2 is a feature which people who is interested in have.
Especially, vehice damage and previously_insured are so important to predict. people have vehicle_damage or previously_insured are subject to be interested in the insurance.

In [ ]:
n_trainV.columns[-1]

In [ ]:
fig=plt.figure(figsize=(15,8))
ax1 = fig.add_subplot(1,2,1)
sns.boxplot(x='Age',y='Response',data=n_trainV)
ax2 = fig.add_subplot(1,2,2)
sns.violinplot(x='Age',y='Response',data=n_trainV)
fig.show()

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(1,2,1)
sns.boxplot(x='Age',y='Response', data=n_trainV)
plt.subplot(1,2,2)
sns.violinplot(x='Age',y='Response', data=n_trainV)
plt.show()

In [ ]:
x2 = n_train.drop(['Response'],1)
y2 = n_train['Response']

In [ ]:
x_tr, x_te, y_tr, y_te = train_test_split(x2, y2, test_size=0.2, random_state=5)

In [ ]:
y_tr.sum() / y_tr.count(), y_te.sum() / y_te.count()

In [ ]:
lj = LogisticRegression(solver='liblinear').fit(x_tr, y_tr)
gnb = GaussianNB().fit(x_tr,y_tr)
knnc = KNeighborsClassifier().fit(x_tr, y_tr)
cartc = DecisionTreeClassifier(random_state=32).fit(x_tr, y_tr)
rfc = RandomForestClassifier(random_state=32, verbose=False).fit(x_tr, y_tr)
gbmc = GradientBoostingClassifier(verbose=False).fit(x_tr, y_tr)
xgbc = XGBClassifier().fit(x_tr, y_tr)
lgbmc = LGBMClassifier().fit(x_tr, y_tr)
catbc = CatBoostClassifier(verbose=False).fit(x_tr, y_tr)

In [ ]:
modelsc = [lj, gnb, knnc, cartc, gbmc, xgbc, lgbmc, catbc]

In [ ]:
def evaluation_models(lists):
    for model in lists:
        name = model.__class__.__name__
        prediction = model.predict(x_te)
        R2CV = cross_val_score(model, x_te, y_te, cv=10, verbose=False).mean()
        error = -cross_val_score(model, x_te, y_te, cv=10, scoring='neg_mean_squared_error', verbose=False).mean()
        print(name+':')
        print('-'*10)
        print(accuracy_score(y_te, prediction))
        print(R2CV)
        print(np.sqrt(error))
        print('-'*30)

In [ ]:
evaluation_models(modelsc)

In [ ]:
def visualize_R2CV(lists):
    df = pd.DataFrame(columns=['MODELS', 'R2CV'])
    for model in lists:
        name = model.__class__.__name__
        R2CV = cross_val_score(model, x_te, y_te, cv=10, verbose=False).mean()
        result = pd.DataFrame([[name,R2CV*100]], columns=['MODELS', 'R2CV'])
        df = df.append(result)
    
    plt.figure(figsize=(10,8))
    sns.barplot(x='R2CV', y='MODELS', data=df, color='blue')
    plt.xlabel('R2CV', fontsize=18)
    plt.ylabel('MODELS', fontsize=18)
    plt.xlim(0,100)
    plt.title('MODEL ACCURACY COMPARISON', fontsize=18)
    plt.show()

In [ ]:
visualize_R2CV(modelsc)

Let's create prediction!

In [ ]:
test_data = pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv') 
test_data = change_object_to_int(test_data)
test_data['Vehicle_Age'] = test_data['Vehicle_Age'].map(mapping)

In [ ]:
test_data

In [ ]:
x3 = test_data.drop(['id'],1)
predict = gbmc.predict(x3)
submission = pd.read_csv('../input/health-insurance-cross-sell-prediction/sample_submission.csv')
submission['Response'] = predict
submission.to_csv('sample_submission.csv', index=False)

In [ ]:
submission

In [ ]:
def visualize_R2CV_2(lists):
    df = pd.DataFrame(columns=['MODELS', 'R2CV'])
    for model in lists:
        name = model.__class__.__name__
        R2CV = cross_val_score(model, x, y, cv=10, verbose=False).mean()
        result = pd.DataFrame([[name,R2CV*100]], columns=['MODELS', 'R2CV'])
        df = df.append(result)
    
    plt.figure(figsize=(10,8))
    sns.barplot(x='R2CV', y='MODELS', data=df, color='blue')
    plt.xlabel('R2CV', fontsize=18)
    plt.ylabel('MODELS', fontsize=18)
    plt.xlim(0,100)
    plt.title('MODEL ACCURACY COMPARISON', fontsize=18)
    plt.show()

In [ ]:
visualize_R2CV_2(modelsc)

SMOTE でデータの不均衡や不足を補う水増しをすることができる
https://qiita.com/ps010/items/38880fad0b8e71464a54

In [ ]:
tr_data2 = tr_data.copy()

In [ ]:
tr_data2

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
X = tr_data2.drop(['Response'],1)
Y = tr_data2['Response']

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.2, random_state=37)

In [ ]:
#一回目はデータをアンダーサンプリングしたので、今回はオーバーサンプリングを試みる

In [ ]:
oversample = SMOTE()
xrTrain, yrTrain = oversample.fit_resample(xTrain, yTrain)

In [ ]:
yrTrain.sum(), yTrain.sum() #オーバーサンプリングされているのが分かる

In [ ]:
lj = LogisticRegression(solver='liblinear').fit(xrTrain, yrTrain)
gnb = GaussianNB().fit(xrTrain,yrTrain)
knnc = KNeighborsClassifier().fit(xrTrain, yrTrain)
cartc = DecisionTreeClassifier(random_state=32).fit(xrTrain, yrTrain)
rfc = RandomForestClassifier(random_state=32, verbose=False).fit(xrTrain, yrTrain)
gbmc = GradientBoostingClassifier(verbose=False).fit(xrTrain, yrTrain)
xgbc = XGBClassifier().fit(xrTrain, yrTrain)
lgbmc = LGBMClassifier().fit(xrTrain, yrTrain)
catbc = CatBoostClassifier(verbose=False).fit(xrTrain, yrTrain)

In [ ]:
def visualize_R2CV_3(lists):
    df = pd.DataFrame(columns=['MODELS', 'R2CV'])
    for model in lists:
        name = model.__class__.__name__
        R2CV = cross_val_score(model, xTest, yTest, cv=10, verbose=False).mean()
        result = pd.DataFrame([[name,R2CV*100]], columns=['MODELS', 'R2CV'])
        df = df.append(result)
    
    plt.figure(figsize=(10,8))
    sns.barplot(x='R2CV', y='MODELS', data=df, color='blue')
    plt.xlabel('R2CV', fontsize=18)
    plt.ylabel('MODELS', fontsize=18)
    plt.xlim(0,100)
    plt.title('MODEL ACCURACY COMPARISON', fontsize=18)
    plt.show()

In [ ]:
visualize_R2CV_3(modelsc)